In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
# Charger les données
df = pd.read_csv("df_with_clusters.csv")
df

,user_number,datetime,Consommation,cluster
0,1,2016-03-01 01:19:43,0,0
1,1,2016-03-01 02:19:43,0,0
2,1,2016-03-01 03:19:43,0,0
3,1,2016-03-01 04:19:43,0,0
4,1,2016-03-01 05:39:54,0,0
...,...,...,...,...
673922,92,2017-02-28 18:08:01,0,2
673923,92,2017-02-28 19:08:06,0,2
673924,92,2017-02-28 20:08:06,27,2
673925,92,2017-02-28 21:08:06,43,2


Préparer les données (fonction de glissement et normalisation)

In [5]:
def create_sliding_window_data(user_data, window_size=5):
    X, y = [], []
    for start in range(0, len(user_data) - window_size):
        end = start + window_size
        X.append(user_data[start:end])
        y.append(user_data[end])  # Prédire la consommation à l'instant suivant
    return np.array(X), np.array(y)

Fonction pour créer et entraîner le modèle LSTM

In [6]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=input_shape))
    model.add(Dense(1))  # Prédiction de la consommation suivante
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Fonction pour entraîner un modèle LSTM pour chaque cluster

In [5]:
def train_lstm_for_cluster(cluster_data, cluster_id, epochs=50, batch_size=32):
    scaler = MinMaxScaler()
    cluster_data_scaled = scaler.fit_transform(cluster_data)
    
    # Diviser les données en ensemble d'entraînement et de validation
    X_train, X_val = train_test_split(cluster_data_scaled, test_size=0.3, random_state=42)

    # Reshape les données pour LSTM (samples, time steps, features)
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

    # Créer et entraîner le modèle LSTM
    model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, X_val))

    # Sauvegarder le modèle LSTM
    model.save(f"LSTM_{cluster_id}.h5")
    
    return history

Fonction pour calculer l'erreur de reconstruction avec le modèle LSTM

In [6]:
def calculate_reconstruction_error_lstm(model, data, scaler):
    data_scaled = scaler.transform(data)
    data_scaled = data_scaled.reshape((data_scaled.shape[0], 1, data_scaled.shape[1]))
    
    reconstructed = model.predict(data_scaled, verbose=0)
    mse = np.mean(np.power(data - reconstructed, 2), axis=1)
    return mse

Entraîner un modèle LSTM pour chaque cluster

In [ ]:
clusters = final_data['cluster'].unique()
for cluster_id in clusters:
    cluster_data = final_data[final_data['cluster'] == cluster_id].drop(columns=['user_number', 'anomalie', 'cluster'])
    
    # Normaliser et entraîner le modèle
    print(f"Entraînement du modèle LSTM pour le cluster {cluster_id}")
    train_lstm_for_cluster(cluster_data, cluster_id)
